In [ ]:
from PIL import Image, ImageDraw
import os
from IPython.display import display
import noise  # pip install noise
import numpy as np
import random

# Parameters for tile generation
TILE_SIZE = 256 # output image size in pixels
CUBE_HEIGHT = 128  # vertical height of the cube in pixels
OUTPUT_DIR = "./"
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
NOISE_SCALE = 2
NOISE_GRANU = 6

def generate_face_perlin(width, height, colors=[(0,255,0)]):
    img = Image.new("RGB", (width, height))
    # random offsets for x and y to add variation
    x_offset = random.uniform(0, 100)
    y_offset = random.uniform(0, 100)
    
    for y in range(height):
        for x in range(width):
            n = noise.pnoise2((x + x_offset) / NOISE_SCALE, 
                              (y + y_offset) / NOISE_SCALE, 
                              octaves=NOISE_GRANU)
            idx = int((n + 0.5) * (len(colors)-1))
            idx = max(0, min(len(colors)-1, idx))
            img.putpixel((x, y), colors[idx])
    return img


In [ ]:
terrain_palette = {
    "ocean": [
        (0, 25, 51),
        (2, 49, 96),
        (0, 68, 136),
        (3, 83, 162),
        (0, 153, 204),
        (102, 221, 238),
    ],
    "lake": [
        (14, 64, 104),
        (24, 88, 141),
        (43, 92, 125),
        (62, 130, 166),
        (112, 175, 194),
        (160, 212, 225),
    ],
    "river": [
        (0, 40, 80),    # deep river (dark navy)
        (0, 60, 120),   # mid-deep blue
        (0, 90, 160),   # medium blue
        (30, 120, 190), # lighter, reflective
        (70, 160, 220), # sunlit/shallow
        (150, 200, 240) # highlight / surface shine
    ],
    "pond": [
        (0, 50, 60),    # deep pond / shadow
        (20, 80, 90),   # mid-deep
        (40, 110, 120), # standard pond
        (70, 140, 150), # shallow / sunlit
        (110, 180, 190),# reflective surface
        (160, 220, 230) # highlight / edges
    ],

    "beach": [
        (253, 245, 230),
        (252, 231, 199),
        (248, 217, 163),
        (242, 201, 123),
        (230, 179, 92),
        (217, 161, 67),
    ],
    "grassland": [
        (44, 95, 45),
        (62, 125, 63),
        (94, 168, 107),
        (127, 207, 141),
        (162, 229, 176),
        (196, 242, 204),
    ],
    "dirt": [
        (62, 39, 35),
        (93, 64, 55),
        (121, 85, 72),
        (160, 82, 45),
        (205, 133, 63),
        (222, 184, 135),
    ],
    "mountain": [
        (60, 50, 50),   # base rock / shadow
        (100, 85, 80),  # mid-dark rock
        (130, 110, 100),# standard rock
        (160, 140, 125),# sunlit rock
        (190, 170, 150),# lighter highlight
        (220, 200, 180) # peak / snow-dusted highlight
    ],

    "ice_cap": [
        (200, 220, 230),# icy shadow
        (210, 230, 240),# light ice
        (220, 240, 250),# bright ice
        (230, 245, 255),# reflective ice
        (240, 250, 255),# near-white
        (250, 255, 255) # pure highlight
    ],
    "forest": [
        (20, 40, 20),   # deep shadow / dense undergrowth
        (35, 60, 35),   # dark green / thick trees
        (50, 85, 50),   # standard foliage
        (70, 110, 70),  # mid-light leaves
        (100, 145, 100),# sunlit leaves
        (150, 200, 150) # bright highlights / young foliage
    ],
    "beach": [
        (253, 245, 230),  # very pale dry sand (top layer)
        (252, 231, 199),  # light sand
        (248, 217, 163),  # mid sand
        (242, 201, 123),  # golden sand
        (230, 179, 92),   # wet sand near water
        (217, 161, 67)    # darker wet sand / edges
    ]



}



In [ ]:

# Function to stretch the image along a direction (dx, dy) and crop back to original size
# This creates a 'brush-like' stretching effect while keeping the same dimensions
def stretch_image_in_place(img, dx, dy, factor):
    w, h = img.size
    arr = np.array(img, dtype=np.float32)
    output = np.zeros_like(arr)
    count = np.zeros((h, w), dtype=np.int32)

    # Normalize direction vector
    length = (dx**2 + dy**2)**0.5
    if length == 0:
        return img
    ux, uy = dx/length, dy/length

    max_steps = int(factor * max(w, h))

    for y in range(h):
        for x in range(w):
            color = arr[y, x]
            for step in range(max_steps):
                new_x = int(x + ux * step)
                new_y = int(y + uy * step)
                if 0 <= new_x < w and 0 <= new_y < h:
                    output[new_y, new_x] += color
                    count[new_y, new_x] += 1
                else:
                    break

    # Average overlapping pixels
    mask = count > 0
    output[mask] = output[mask] / count[mask][:, None]

    return Image.fromarray(output.astype(np.uint8))


In [ ]:
# Function to generate a cube tile with customizable colors for each face
def generate_cube_tile_faces(cube_height=CUBE_HEIGHT,
                             top_image: Image.Image | None = None,
                             top_colors  :list[tuple[int,int,int]]=terrain_palette["grassland"],
                             left_colors :list[tuple[int,int,int]]=terrain_palette["dirt"],
                             right_colors:list[tuple[int,int,int]]=terrain_palette["dirt"],
                             filename="cube_tile_custom.png",
                             stretch_dir:list[tuple[float,float,float]] = [(0,0,0)]):
    image = Image.new("RGB", (TILE_SIZE, TILE_SIZE), (0, 0, 0))

    cx = TILE_SIZE // 2
    cy = TILE_SIZE // 2
    half_width = TILE_SIZE // 2
    half_height = TILE_SIZE // 4

    # Define faces
    top = [(cx - half_width, cy - half_height), (cx, cy - half_height*2), (cx + half_width, cy - half_height), (cx, cy)]
    left = [top[0], top[3], (top[3][0], top[3][1] + cube_height), (top[0][0], top[0][1] + cube_height)]
    right = [top[2], top[3], (top[3][0], top[3][1] + cube_height), (top[2][0], top[2][1] + cube_height)]

    # Masks
    top_mask = Image.new("L", (TILE_SIZE, TILE_SIZE), 0)
    ImageDraw.Draw(top_mask).polygon(top, fill=255)
    left_mask = Image.new("L", (TILE_SIZE, TILE_SIZE), 0)
    ImageDraw.Draw(left_mask).polygon(left, fill=255)
    right_mask = Image.new("L", (TILE_SIZE, TILE_SIZE), 0)
    ImageDraw.Draw(right_mask).polygon(right, fill=255)

    # Generate Perlin faces
    top_img = generate_face_perlin(TILE_SIZE, TILE_SIZE, colors=top_colors)
    left_img = generate_face_perlin(TILE_SIZE, TILE_SIZE, colors=left_colors)
    right_img = generate_face_perlin(TILE_SIZE, TILE_SIZE, colors=right_colors)

    top_img   = stretch_image_in_place(top_img, stretch_dir[0][0], stretch_dir[0][1], stretch_dir[0][2])

    # Paste faces
    if top_image:
        top_img = top_image

    if len(stretch_dir) > 1:
        left_img  = stretch_image_in_place(left_img,  stretch_dir[1][0], stretch_dir[1][1], stretch_dir[1][2])
    if len(stretch_dir) > 2:
        right_img = stretch_image_in_place(right_img, stretch_dir[2][0], stretch_dir[2][1], stretch_dir[2][2])

    # Paste faces
    image.paste(top_img, (0,0), top_mask)
    image.paste(left_img, (0,0), left_mask)
    image.paste(right_img, (0,0), right_mask)

    # Draw edges for lateral faces
    draw = ImageDraw.Draw(image)
    # draw.line(left + [left[0]], fill=(0,0,0), width=1)
    # draw.line(right + [right[0]], fill=(0,0,0), width=1)

    save_path = os.path.join(OUTPUT_DIR, filename)
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    image.save(save_path, "PNG")
    # display(image)


In [ ]:
from PIL import Image, ImageDraw
import random, noise
import math

def generate_grass():
    SIZE = 256
    img = Image.new("RGB", (SIZE, SIZE), (34, 80, 0))
    draw = ImageDraw.Draw(img)

    # --- Step 1: base noise shading ---
    for y in range(SIZE):
        for x in range(SIZE):
            n = noise.pnoise2(x/50, y/50, octaves=3)
            shade = int(40 * n)
            r, g, b = img.getpixel((x, y))
            draw.point((x, y), (
                max(0, r+shade),
                min(255, g+shade),
                max(0, b+shade)
            ))

    # --- Step 2: draw blades ---
    for _ in range(10000):  # more blades for detail
        x = random.randint(0, SIZE-1)
        y = random.randint(0, SIZE-1)
        length = random.randint(5, 12)   # varied blade sizes
        curve = random.uniform(-0.6, 0.6)

        # base blade color
        base_green = random.randint(60, 180)
        tip_shift = random.randint(20, 50)

        # draw blade as small segments (to curve it)
        px, py = x, y
        for i in range(length):
            # color gradient along blade
            color = (20, min(200, base_green + i*2), 20 + i)

            # slight curve in x direction
            nx = px + curve
            ny = py - 1

            draw.line((px, py, nx, ny), fill=color, width=1)
            px, py = nx, ny

    # # --- Step 3: add subtle soil specks ---
    # for _ in range(150):
    #     x = random.randint(0, SIZE-1)
    #     y = random.randint(0, SIZE-1)
    #     if random.random() < 0.5:
    #         draw.point((x, y), (80, 50, 20))  # brown speck

    return img

In [ ]:

# Example usage
n_of_samples = 10
for i in range(n_of_samples):
    grass = generate_grass()
    NOISE_SCALE = 10
    generate_cube_tile_faces(CUBE_HEIGHT, grass,terrain_palette["grassland"], terrain_palette["dirt"], terrain_palette["dirt"], f"terrains/grassland/tile_{i:02d}.png")
    generate_cube_tile_faces(CUBE_HEIGHT, None, terrain_palette["forest"], terrain_palette["dirt"], terrain_palette["dirt"], f"terrains/forest/tile_{i:02d}.png")
    generate_cube_tile_faces(CUBE_HEIGHT, None, terrain_palette["mountain"], terrain_palette["mountain"], terrain_palette["mountain"], f"terrains/mountain/tile_{i:02d}.png")
    generate_cube_tile_faces(CUBE_HEIGHT, None, terrain_palette["ice_cap"], terrain_palette["ice_cap"], terrain_palette["ice_cap"], f"terrains/ice_cap/tile_{i:02d}.png")
    generate_cube_tile_faces(CUBE_HEIGHT, None, terrain_palette["beach"], terrain_palette["beach"], terrain_palette["beach"], f"terrains/beach/tile_{i:02d}.png")


    NOISE_SCALE = 20
    generate_cube_tile_faces(CUBE_HEIGHT//2, None, terrain_palette["river"], terrain_palette["river"], terrain_palette["river"], f"terrains/river/tile_{i:02d}.png")
    generate_cube_tile_faces(CUBE_HEIGHT//2, None, terrain_palette["lake"],  terrain_palette["lake"],  terrain_palette["lake"],  f"terrains/lake/tile_{i:02d}.png")
    generate_cube_tile_faces(CUBE_HEIGHT//2, None, terrain_palette["ocean"], terrain_palette["ocean"], terrain_palette["ocean"], f"terrains/ocean/tile_{i:02d}.png")
    generate_cube_tile_faces(CUBE_HEIGHT//2, None, terrain_palette["pond"], terrain_palette["pond"], terrain_palette["pond"], f"terrains/pond/tile_{i:02d}.png")
    # , [(1,0.5,0.1), (1,0.5,0.1)]

In [ ]:
grid = np.zeros((10,10))
# counter = 0
# for x in range(10):
#     for y in range(10):
#         grid[y,x] = counter
#         counter += 1
i, j = 2,2
grid[i+0,j+0] = 1
grid[i+0,j+1] = 2
grid[i+1,j+0] = 2
grid[i+1,j+1] = 3
grid[i+0,j+2] = 3
grid[i+2,j+0] = 3
grid[i+2,j+1] = 4
grid[i+1,j+2] = 4
grid[i+0,j+3] = 4
grid[i+3,j+0] = 4


print(grid)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 2. 3. 4. 0. 0. 0. 0.]
 [0. 0. 2. 3. 4. 0. 0. 0. 0. 0.]
 [0. 0. 3. 4. 0. 0. 0. 0. 0. 0.]
 [0. 0. 4. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
